In [1]:
!pip install -qq datasets evaluate transformers[sentencepiece] scattertext emoji

     |████████████████████████████████| 451 kB 5.2 MB/s 
     |████████████████████████████████| 72 kB 1.0 MB/s 
     |████████████████████████████████| 5.5 MB 41.4 MB/s 
     |████████████████████████████████| 7.3 MB 6.9 MB/s 
     |████████████████████████████████| 240 kB 55.1 MB/s 
     |████████████████████████████████| 182 kB 51.6 MB/s 
     |████████████████████████████████| 212 kB 46.8 MB/s 
     |████████████████████████████████| 115 kB 57.1 MB/s 
     |████████████████████████████████| 127 kB 33.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.7 MB/s 
     |████████████████████████████████| 7.6 MB 34.8 MB/s 
     |████████████████████████████████| 1.3 MB 68.5 MB/s 


In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import transformers
import re
import emoji
from transformers import AutoTokenizer
from tqdm import tqdm
tqdm.pandas()
from datasets import Dataset, load_dataset, load_metric

print(transformers.__version__)

4.24.0


In [5]:
data = pd.read_excel("/content/df_sentiment_week2.xlsx")

In [6]:
data.head(1)

,date,username,tweet,pysentiment
0,"Wednesday 21 Sep 2022, 23:59:57",perbowo,user user user thats mah bjorka kw 13,1.0


In [7]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [8]:
data['pysentiment'] = data['pysentiment'].replace({2:'Positif',1:'Netral',0:'Negatif'})

In [9]:
data = data.dropna()

In [10]:
from scattertext import CorpusFromPandas, produce_scattertext_explorer
import scattertext as st

print("Document Count")
scattext_df = data.copy()
scattext_df = scattext_df[scattext_df.pysentiment != "Netral"]
print(scattext_df.groupby('pysentiment')['tweet'].count())

Document Count
pysentiment
Negatif    11859
Positif     3978
Name: tweet, dtype: int64


In [25]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words=stopwords.words('english')
stop_words.extend(['https','co','bjorka','like','know','really','still','user','smiling','face','eye',
                   'beaming','emoji','face','welcome','hearteyes','httpurl','wkwkwk'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
scattext_df['tweet'] = scattext_df.tweet.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [27]:
print("Word Count")
scattext_df.groupby('pysentiment').apply(lambda x: x.tweet.apply(lambda x: len(tokenize(x))).sum())
scattext_df['parsed'] = scattext_df.tweet.apply(st.whitespace_nlp_with_sentences)

Word Count


In [28]:
# membuat corpus untuk scatter text 
corpus = st.CorpusFromParsedDocuments(scattext_df, category_col = 'pysentiment', \
                                      parsed_col = 'parsed').build()

In [29]:


html = produce_scattertext_explorer(corpus,
                                    category = 'Positif',
                                    category_name = 'Positif',
                                    not_category_name = 'Negatif',
                                    width_in_pixels = 1000,
                                    jitter = 0.1,
                                    minimum_term_frequency = 5,
                                    transform = st.Scalers.percentile)

file_name = 'LabelPemerintahScattertextRankDataJitter.html'
open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src = file_name, width = 1200, height = 700)

3402124

In [ ]:
whole_df = pd.read_excel("/content/df_pemerintah.xlsx")
whole_df = whole_df.fillna(0)
whole_df = whole_df[['tweet','pysentiment']]
whole_df['pysentiment'] = whole_df['pysentiment'].astype(int)
whole_df.columns = ['text','label']
dataset = Dataset.from_pandas(whole_df, preserve_index=False).train_test_split(test_size=0.2)
dataset

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 264
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 66
    })
})

In [ ]:
PRETRAINED_NAME = "w11wo/indonesian-roberta-base-sentiment-classifier"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_NAME)

Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(ex):
  return tokenizer(ex["text"], padding="max_length", truncation=True, max_length=512)  

tokenized_dataset = dataset.map(tokenize_fn, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average="macro")

f1_metric = evaluate.load("f1", average="macro")
clf_metrics = evaluate.combine(["accuracy", f1_metric])

In [ ]:
from transformers import pipeline
from sklearn.metrics import classification_report
w11wo_clf = pipeline(
  "sentiment-analysis",
  model="w11wo/indonesian-roberta-base-sentiment-classifier",
  tokenizer="w11wo/indonesian-roberta-base-sentiment-classifier",
  device=0
)

model = w11wo_clf.model

# freeze
for param in model.roberta.parameters():
  param.requires_grad = False

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--w11wo--indonesian-roberta-base-sentiment-classifier/snapshots/b359aedef1ff88b64a47b2378cb542ef037bfc49/config.json
Model config RobertaConfig {
  "_name_or_path": "w11wo/indonesian-roberta-base-sentiment-classifier",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "neutral",
    "2": "negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 2,
    "neutral": 1,
    "positive": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_typ

In [ ]:
training_args = TrainingArguments(output_dir="sentiment_tuned", 
                                  evaluation_strategy="epoch", num_train_epochs=10, 
                                  learning_rate=1e-5, per_device_eval_batch_size=32,
                                  per_device_train_batch_size=8,metric_for_best_model = "eval_loss")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 264
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 330
  Number of trainable parameters = 592899


Epoch,Training Loss,Validation Loss,F1
1,No log,0.514099,0.749288
2,No log,0.514149,0.749288
3,No log,0.522539,0.766234
4,No log,0.513855,0.766234
5,No log,0.506421,0.766234
6,No log,0.508531,0.766234
7,No log,0.505398,0.766234
8,No log,0.504016,0.766234
9,No log,0.504782,0.766234
10,No log,0.503816,0.766234


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 32
T

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 32


{'eval_loss': 0.5038161873817444,
 'eval_f1': 0.7662337662337662,
 'eval_runtime': 2.1227,
 'eval_samples_per_second': 31.092,
 'eval_steps_per_second': 1.413,
 'epoch': 10.0}

In [ ]:
pred_test = [v for v in np.argmax(trainer.predict(tokenized_dataset["test"]).predictions, axis=1)]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 66
  Batch size = 32


In [ ]:
print(classification_report(dataset["test"]["label"], pred_test, digits=4))

              precision    recall  f1-score   support

           0     0.9118    0.7209    0.8052        43
           1     0.6250    0.8696    0.7273        23

    accuracy                         0.7727        66
   macro avg     0.7684    0.7952    0.7662        66
weighted avg     0.8118    0.7727    0.7780        66



### w11wo/indonesian-roberta-base-sentiment-classifier

In [ ]:
from transformers import pipeline
from sklearn.metrics import classification_report

In [ ]:
w11wo_clf = pipeline(
  "sentiment-analysis",
  model="w11wo/indonesian-roberta-base-sentiment-classifier",
  tokenizer="w11wo/indonesian-roberta-base-sentiment-classifier",
  device=0
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--w11wo--indonesian-roberta-base-sentiment-classifier/snapshots/b359aedef1ff88b64a47b2378cb542ef037bfc49/config.json
Model config RobertaConfig {
  "_name_or_path": "w11wo/indonesian-roberta-base-sentiment-classifier",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "neutral",
    "2": "negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 2,
    "neutral": 1,
    "positive": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_typ

In [ ]:
dict_label = {"negative":0,'neutral':1,'positive':2}

In [ ]:
res_w11wo_clf = w11wo_clf(dataset["test"]["text"])
pred_w11wo_clf = [dict_label[v["label"]] for v in res_w11wo_clf]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
print(classification_report(dataset["test"]["label"], pred_w11wo_clf, digits=4))

              precision    recall  f1-score   support

           0     0.8974    0.8140    0.8537        43
           1     0.7917    0.8261    0.8085        23
           2     0.0000    0.0000    0.0000         0

    accuracy                         0.8182        66
   macro avg     0.5630    0.5467    0.5541        66
weighted avg     0.8606    0.8182    0.8379        66



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df = pd.read_excel('/content/df_sample_sentiment_pemerintah_week2 (1).xlsx')

In [ ]:
w11wo_clf = pipeline(
  "sentiment-analysis",
  model="w11wo/indonesian-roberta-base-sentiment-classifier",
  tokenizer="w11wo/indonesian-roberta-base-sentiment-classifier",
  device=0
)

In [ ]:
df = df.dropna(subset=['scoring'])

In [ ]:
df['scoring']=df['scoring'].astype(str)

In [ ]:
df.scoring = df['scoring'].str[0]

In [ ]:
print(classification_report(df.pysentiment, df.scoring.astype(int), digits=4))

              precision    recall  f1-score   support

         0.0     0.6629    0.7785    0.7160       149
         1.0     0.7815    0.6860    0.7307       172
         2.0     0.2500    0.1111    0.1538         9

    accuracy                         0.7121       330
   macro avg     0.5648    0.5252    0.5335       330
weighted avg     0.7134    0.7121    0.7083       330



In [ ]:
df.scoring.str.isnumeric()

0       True
1       True
3       True
4       True
5      False
       ...  
491     True
492     True
493     True
494     True
495     True
Name: scoring, Length: 408, dtype: bool

In [ ]:
df = df[df.scoring.str.isnumeric()]

In [ ]:
pred = df.tweet.apply(lambda x: w11wo_clf(x))
preds = [dict_label[v[0]["label"]] for v in pred.values]

In [ ]:
df

,date,username,tweet,pysentiment,scoring,Unnamed: 5
0,"Thursday 15 Sep 2022, 04:09:30",Jizzy_bunsss,@memefess Gw ga nyembah dia tp gw bingung sm b...,1.0,1,NaN
1,"Sunday 18 Sep 2022, 10:40:16",midasbuy_id,Sekarang Bjorka mau ngebacot soal Muhaimin PKB...,0.0,1,NaN
3,"Friday 16 Sep 2022, 11:33:43",fadelyafie,Ada gak sih yg kontra hacker Bjorka selain pem...,0.0,0,NaN
4,"Monday 19 Sep 2022, 04:14:50",nynynim,How it should be: \nBjorka meningkatkan awaren...,2.0,2,NaN
7,"Thursday 15 Sep 2022, 14:58:24",Asep_Ac023,Lanjutkan terus @bjorka tegakan keadilan apa s...,0.0,0,NaN
...,...,...,...,...,...,...
491,"Thursday 15 Sep 2022, 13:00:53",mas_ngafif,"Apa ndak malu ya, sdh berapa kali salah tangka...",0.0,0,NaN
492,"Thursday 15 Sep 2022, 09:39:53",bininyakjongin,@BuAgus17 @mayfunnie @detikcom Bagus dong gitu...,0.0,0,NaN
493,"Saturday 17 Sep 2022, 13:30:59",hahanying15,Banyak yang bilang Bjorka ini pengalihan isu k...,0.0,0,NaN
494,"Thursday 15 Sep 2022, 06:23:04",katakinidotcom,Diduga Terkait Peretasan Bjorka Polisi Tangkap...,1.0,1,NaN


In [ ]:
df.scoring.values

array(['1', '1', '0', '2', '0', '0', '0', '0', '1', '1', '1', '1', '1',
       '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0',
       '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0',
       '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0',
       '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1',
       '0', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '1', '1',
       '1', '1', '1', '0', '0', '0', '1', '1', '1', '0', '1', '1', '1',
       '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1',
       '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '2', '1',
       '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1',
       '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0',
       '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0

In [ ]:
print(classification_report(df.scoring.astype(int).values, preds, digits=4))

              precision    recall  f1-score   support

           0     0.6762    0.8114    0.7377       175
           1     0.8020    0.5364    0.6429       151
           2     0.0000    0.0000    0.0000         4

    accuracy                         0.6758       330
   macro avg     0.4927    0.4493    0.4602       330
weighted avg     0.7256    0.6758    0.6853       330

